<a href="https://colab.research.google.com/github/sonalgan/DeepLearning/blob/main/02HPE_DL_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Downloading Dataset


In [ ]:
!wget -q http://lotus.kuee.kyoto-u.ac.jp/WAT/indic-multilingual/indic_wat_2021.tar.gz
!tar xzf indic_wat_2021.tar.gz
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git -q
!git clone https://github.com/anoopkunchukuttan/indic_nlp_library.git -q
!mkdir data

fatal: destination path 'indic_nlp_resources' already exists and is not an empty directory.
fatal: destination path 'indic_nlp_library' already exists and is not an empty directory.


In [ ]:
punctuation=[",","।",";","?","!","—","‐",":-","'","(",")",'"']

In [ ]:
languages=["hi","bn","gu","kn","mr","or","ta","te","ml","pa"]
languages.sort()

In [ ]:
import sys
# The path to the local git repo for Indic NLP library
INDIC_NLP_LIB_HOME=r"/content/indic_nlp_library"
# The path to the local git repo for Indic NLP Resources
INDIC_NLP_RESOURCES=r"/content/indic_nlp_resources"
sys.path.append(r'{}'.format(INDIC_NLP_LIB_HOME))
from indicnlp import common
common.set_resources_path(INDIC_NLP_RESOURCES)
from indicnlp import loader
loader.load()


In [ ]:
import string
for char in string.punctuation:
  print(char,end=" ")
translate_table=dict((ord(char),None) for char in string.punctuation) 
global translate_table

! " # $ % & ' ( ) * + , - . / : ; < = > ? @ [ \ ] ^ _ ` { | } ~ 

In [ ]:
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
from indicnlp.tokenize import sentence_tokenize 
from re import sub as resub
import csv

factory=IndicNormalizerFactory()

Pipeline for Tokenizing the sentences while removing special char, or roman script


In [ ]:

class LangCleaner:
  def __init__(self):
    self.lang="hi"
    self.normalizer=None

  def loader(self,lang):
    self.lang=lang
    self.fields=["sentences","length"] 
    
    fp = "finalrepo/train/pmi/en-{}/train.{}".format(lang,lang)
    with open(fp) as f:
      chunk = f.readlines()
      self.normalizer=factory.get_normalizer(lang)
    return chunk
  def cleaner(self,chunk):
    vocab=[]
    len_sentence=[]
    for line in chunk:
      line=line.rstrip("\n")
      line=self.normalizer.normalize(line)
      line= resub(r"\d+","",line)
      line=resub(r"[a-zA-Z]+","",line)
      # line=resub(r'[\u0964\u2022\u00B7]','',line)
      line=line.translate(translate_table)
      sentences=sentence_tokenize.sentence_split(line, lang=self.lang)
      for sentence in sentences:
        vocab.append(sentence)
        len_sentence.append(len(sentence))
    with open('data/{}.csv'.format(self.lang), 'w',encoding="utf-8") as outputfile:
      csvwriter=csv.writer(outputfile)
      csvwriter.writerow(self.fields)
      for i, (x, y) in enumerate(zip(vocab, len_sentence)):
        csvwriter.writerow([x,y])
    return len(vocab)
obj=LangCleaner()
for lang in languages:
  ans=obj.loader(lang)
  len_data=obj.cleaner(ans)
  print("Length of",lang,"dataset:",len_data)
del obj
del factory
del loader




Length of bn dataset: 23657
Length of gu dataset: 41578
Length of hi dataset: 50412
Length of kn dataset: 28902
Length of ml dataset: 26916
Length of mr dataset: 28976
Length of or dataset: 32017
Length of pa dataset: 28414
Length of ta dataset: 32638
Length of te dataset: 33380


In [ ]:
'''
import os
filelist = [ f for f in os.listdir("data") ]
for f in filelist:
    os.remove(os.path.join("data",f))
    '''